## This is a debuging ipython notebook

In [1]:
%load_ext autoreload
import argparse
import warnings
warnings.filterwarnings('ignore')

arg_parser = argparse.ArgumentParser()
        
    # # Add standard arguments
    # if not is_standalone:
    #     # If standalone, set in parser.
arg_parser.add_argument('--output_path', default='.',
                                help='[Default is current directory] Path to '
                                     'output directory.')
arg_parser.add_argument('--version', action='version',
                            help='Show programme\'s version number and exit.',
                            version='0.1')

arg_parser.add_argument('--standalone_path', default='/Users/daviddylus/Research/pore2tree/reference_datasets/Dataset1/',
                            help='[Default is current directory] Path to '
                                 'oma standalone directory.', required=True)
arg_parser.add_argument('--remove_species', default=None,
                            help='Remove species present in dataset and only do analysis on '
                                 'subset of species')
arg_parser.add_argument('--keep_all_ogs', action='store_true',
                            help='Keep all orthologs after addition of mapped seq, which means '
                            'also the groups that have no mapped sequence. Otherwise only '
                            'groups are used that have the mapped sequence for alignment '
                            'and tree inference.')

arg_parser.add_argument('--ref_og_aa_folder', default='.',
                            help='Path to preselected og_aa folder')

arg_parser.add_argument('--single_mapping', default=None,
                            help='Single species file allowing to map in a job array.')

arg_parser.add_argument('--threads', type=int, default=None,
                            help='Number of threads for the mapping using ngm / ngmlr!')

arg_parser.add_argument('--remove_species_mapping_only', action='store_true',
                            help='[Default is remove from everywhere]'
                                 'Remove species only from mapping set.')

    # Arguments to generate the reference
arg_parser.add_argument('-r', '--reference', action='store_true',
                            help='Just generate the reference dataset for mapping')
arg_parser.add_argument('--min_species', type=int, default=30,
                            help='Min number of species in selected orthologous groups. \
                            If not selected it will be estimated such that around 1000 OGs are available.')
arg_parser.add_argument('--dna_reference', default='/Volumes/Untitled/eukaryotes.cdna.fa',
                            help='Reference fasta file that contains nucleotide sequences.')

    # Arguments to map the reads
arg_parser.add_argument('--ref_folder', default=None,
                            help='Folder containing reference files with sequences sorted by species.')
arg_parser.add_argument('--reads', default='/Users/daviddylus/Research/pore2tree/fritz_scripts/pipeline/Nano_reads/ERR1877969.fastq',
                            help='Reads to be mapped to reference.')
# arg_parser.add_argument('--reads', nargs='2',default=None, help='Reads to be mapped to reference.')


# Parse the arguments.
# reference = ['--ref_folder', '/Users/daviddylus/Research/pore2tree/pore2tree/tests/mapper/test1/db/']
argv = ['--output_path','/Users/daviddylus/Desktop/bla_testd/','--standalone_path', '/Users/daviddylus/Research/read2tree/read2tree/tests/oma_standalone/', '--min_species', 
        '5', '--dna_reference', '/Users/daviddylus/mnt/omabrowser/OmaServer.h5', '--remove_species', 'CANAW,YEAST',
        '--reads', '/Users/daviddylus/Research/read2tree/read2tree/tests/mapper/test3/test_1b.fq /Users/daviddylus/Research/read2tree/read2tree/tests/mapper/test3/test_2b.fq']
args = arg_parser.parse_args(argv)
print(args)

Namespace(dna_reference='/Users/daviddylus/mnt/omabrowser/OmaServer.h5', keep_all_ogs=False, min_species=5, output_path='/Users/daviddylus/Desktop/bla_testd/', reads='/Users/daviddylus/Research/read2tree/read2tree/tests/mapper/test3/test_1b.fq /Users/daviddylus/Research/read2tree/read2tree/tests/mapper/test3/test_2b.fq', ref_folder=None, ref_og_aa_folder='.', reference=False, remove_species='CANAW,YEAST', remove_species_mapping_only=False, single_mapping=None, standalone_path='/Users/daviddylus/Research/read2tree/read2tree/tests/oma_standalone/', threads=None)


In [2]:
from read2tree.Progress import Progress
progress = Progress(args)
print(progress.status)

3


In [3]:
from read2tree.OGSet import OGSet
from read2tree.parser import OMAOutputParser
if progress.status >= 1:
    ogset = OGSet(args, load=False)
else:
    oma_output = OMAOutputParser(args)
    args.oma_output_path = oma_output.oma_output_path
    ogset = OGSet(args, oma_output=oma_output)

Loading files: 5 OGs [00:00, 573.26 OGs/s]

--- Re-load ogs and find their corresponding DNA seq from output folder ---


In [4]:
from read2tree.ReferenceSet import ReferenceSet
if progress.status >= 2:
    reference = ReferenceSet(args, load=False)
else:
    reference = ReferenceSet(args, og_set=ogset.ogs, load=True)

Re-loading references for mapping from folder: 100%|██████████| 20/20 [00:00<00:00, 4740.67 species/s]

--- Generating reference for mapping from folder ---


In [5]:
# %load_ext autoreload
#%autoreload
from read2tree.Mapper import Mapper
if progress.status >= 3:
    mapper = Mapper(args, og_set=ogset.ogs, load=False)
else:
    mapper = Mapper(args, ref_set=reference.ref, og_set=ogset.ogs)

Loading consensus read mappings : 100%|██████████| 6/6 [00:00<00:00, 5381.91 species/s]

--- Retrieve mapped consensus sequences ---


In [6]:
new_ogs = ogset.add_mapped_seq_v2(mapper.og_records)

Adding mapped seq to OG: 100%|██████████| 5/5 [00:00<00:00, 2602.89 OGs/s]

--- Add inferred mapped sequence back to OGs ---


In [ ]:
bla = ['x','a','y','d']
blub = ['y','a','d','a']
rem = ['x','a']
ind = []
for i,letter in enumerate(zip(bla,blub)):
    print(letter[0])
    if letter[0] in rem:
        ind.append(i)
        
print([i for j, i in enumerate(bla) if j not in ind])
# print(zip(bla,blub)[0])

In [ ]:
%reload_ext autoreload 
%autoreload 2 
from read2tree import Aligner
alignments = Aligner.Aligner(args, ogset.mapped_ogs)

In [ ]:
concat_alignment = alignments.concat_alignment()

In [ ]:
from read2tree import TreeInference
tree = TreeInference.TreeInference(args, concat_alignment=concat_alignment)

In [ ]:
print(tree.tree)

In [ ]:
import re
bla = 'bascdd [KADO]'
re.search('^(.*?)\[(.*)\]', '', bla)

In [ ]:
import pyopa
